### Convert mp4 to mp3

In [32]:
from moviepy.editor import VideoFileClip
from pydub import AudioSegment
import os


def convert_mp4_to_mp3(mp4_file, mp3_file):
    # Load the video file
    video_clip = VideoFileClip(mp4_file)

    # Extract the audio
    audio_clip = video_clip.audio

    # Save the audio as a temporary WAV file
    temp_wav_file = "temp.wav"
    audio_clip.write_audiofile(temp_wav_file)

    # Convert the WAV file to MP3
    audio = AudioSegment.from_wav(temp_wav_file)
    audio.export(mp3_file, format="mp3")

    # Remove the temporary WAV file
    os.remove(temp_wav_file)

In [2]:
convert_mp4_to_mp3("pwedipie.mp4", "pwedipie.mp3")

MoviePy - Writing audio in temp.wav


MoviePy - Done.


### Write text on video

In [58]:
import cv2
import moviepy.editor as mpe


def write_text_on_video(input_video_path, output_video_path, transcript):
    # Open the video file
    video = cv2.VideoCapture(input_video_path)

    # Get the video's resolution and FPS
    frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = video.get(cv2.CAP_PROP_FPS)

    # Define the codec and create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    output_video_path_temp = output_video_path.replace('.mp4', '_no_sound.mp4')
    out = cv2.VideoWriter(output_video_path_temp, fourcc, fps, (frame_width, frame_height))

    starts = [int(el['start'] * fps) for el in transcript]
    ends = [int(el['end'] * fps) for el in transcript]
    words = [el['word'] for el in transcript]

    wordindex = 0

    frame_count = 0
    while video.isOpened():
        # Read the video frame by frame
        ret, frame = video.read()

        if not ret:
            break

        # Define the text properties
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1
        color = (255, 0, 0)  # BGR, not RGB
        thickness = 2

        if starts[wordindex] <= frame_count <= ends[wordindex]:
            frame = cv2.putText(frame, words[wordindex], (50, 50), font, font_scale, color, thickness, cv2.LINE_AA)

        if(frame_count > ends[wordindex] and wordindex != (len(starts)-1)):
            print(wordindex)
            wordindex  = wordindex + 1 if(wordindex != (len(starts)-1)) else wordindex

        # Write the modified frame to the output video file
        out.write(frame)

        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        frame_count += 1

    # Release everything when done

    video.release()
    out.release()
    cv2.destroyAllWindows()
    # replace the audio

    convert_mp4_to_mp3(input_video_path, 'input.mp3')
    my_clip = mpe.VideoFileClip(output_video_path_temp)
    audio_background = mpe.AudioFileClip('input.mp3')
    final_clip = my_clip.set_audio(audio_background)
    final_clip.write_videofile(output_video_path)
    
    os.remove(output_video_path_temp)

In [59]:
transcript = [
    {'start': 1, 'end': 2, 'word': 'Hello,'},
    {'start': 2, 'end': 4, 'word': 'World!'},
    {'start': 5, 'end': 6, 'word': 'This'},
    {'start': 6, 'end': 7, 'word': 'is'},
    {'start': 7, 'end': 8, 'word': 'a'},
    {'start': 8, 'end': 9, 'word': 'test.'}
]

write_text_on_video('input.mp4', 'output_video.mp4', transcript)

0
1
2
3
4
MoviePy - Writing audio in temp.wav


MoviePy - Done.


Moviepy - Building video output_video.mp4.
MoviePy - Writing audio in output_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output_video.mp4



Moviepy - Done !
Moviepy - video ready output_video.mp4


In [60]:
import json

transcript = json.load(open('output_pwedipie.txt'))['words']
print(transcript)

write_text_on_video('pwedipie.mp4', 'pwedipie_subtitles.mp4', transcript)

[{'word': 'Slinky', 'start': 0.36000001430511475, 'end': 0.9200000166893005}, {'word': 'you', 'start': 1.059999942779541, 'end': 1.559999942779541}, {'word': 'know', 'start': 1.559999942779541, 'end': 1.6799999475479126}, {'word': 'that', 'start': 1.6799999475479126, 'end': 2.1600000858306885}, {'word': 'thing', 'start': 2.1600000858306885, 'end': 2.4600000381469727}, {'word': 'that', 'start': 2.4600000381469727, 'end': 3.0}, {'word': 'goes', 'start': 3.0, 'end': 3.1600000858306885}, {'word': 'down', 'start': 3.1600000858306885, 'end': 3.359999895095825}, {'word': 'the', 'start': 3.359999895095825, 'end': 3.9800000190734863}, {'word': 'stairs', 'start': 3.9800000190734863, 'end': 3.9800000190734863}, {'word': 'How', 'start': 4.039999961853027, 'end': 4.599999904632568}, {'word': 'does', 'start': 4.599999904632568, 'end': 4.800000190734863}, {'word': 'he', 'start': 4.800000190734863, 'end': 5.0}, {'word': 'know', 'start': 5.0, 'end': 5.199999809265137}, {'word': 'how', 'start': 5.199999

MoviePy - Done.
Moviepy - Building video pwedipie_subtitles.mp4.
MoviePy - Writing audio in pwedipie_subtitlesTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video pwedipie_subtitles.mp4



Moviepy - Done !
Moviepy - video ready pwedipie_subtitles.mp4


### Get text of video


In [1]:
from openai import OpenAI
import os

api_key = os.getenv("OPENAI_API_KEY_LORRAIN")
# print(api_key)
if api_key is None:
    raise ValueError("OPENAI_API_KEY environment variable not set")

client = OpenAI(api_key=api_key)

def getTextOfVideo(mp3file):
    audio_file = open(mp3file, "rb")
    transcript = client.audio.transcriptions.create(
        file=audio_file,
        model="whisper-1",
        response_format="verbose_json",
        timestamp_granularities=["word"]
    )

    return transcript



In [4]:
transcrpt = getTextOfVideo("pwedipie.mp3")
print(transcrpt)
print(transcrpt.words)

Transcription(text="Slinky, you know that thing that goes down the stairs. How does he know how to do that? These are pretty cool. But other than that, there's not much you can do with it. How wrong I was Chinese man studied the slinky while man in West have been busy diddly-daddling Chinese man have perfected the slinky It is shot that kid in the head. It is decided I will master the art of bending the slinky to my own will I'm gonna practice the slinky for two weeks. How hard can he be day one? I got my slinky. I Didn't really know how to use it, but I knew it would make Bjorn laugh I Decided I'm gonna master the slinky to once again impress my son with an epic performance That will blow his mind day two. I practice the slinky shot Day three, I try to learn the hyper beam. How does it work? I Move it from one to another I was not getting it Day five I started practicing throwing the slinky You may have noticed I am NOT a slinky prodigy just yet just just Your mind My philosophy is it